# Player Season Stats Modeling – IPL

This notebook ingests historical IPL player season-level statistics,
normalizes the schema, and models the data into a fact table
(`fact_player_season_stats`) for downstream analytics and fantasy projections.


In [10]:
import pandas as pd
from datetime import datetime
import hashlib

In [18]:
DATA_SOURCE = "IPL Player Season Stats CSV"
INGESTION_DATE = datetime.now().isoformat()
RAW_FILE_PATH = "../data/processed/cricket_data_2025.csv"

In [19]:
df_raw = pd.read_csv(RAW_FILE_PATH)
df_raw.head()

,Year,Player_Name,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,Batting_Strike_Rate,Centuries,...,Matches_Bowled,Balls_Bowled,Runs_Conceded,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls
0,NaN,Aaron Hardie,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,...,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats
1,2024.0,Abdul Samad,16,2,182,37*,18.2,108,168.52,0,...,16,0,0,0,0,0,0,0,0,0
2,2023.0,Abdul Samad,9,4,169,37*,42.25,128,132.03,0,...,9,0,0,0,0,0,0,0,0,0
3,2022.0,Abdul Samad,2,0,4,4,2.0,7,57.14,0,...,2,6,8,0,0/8,0,8.00,0,0,0
4,2021.0,Abdul Samad,11,1,111,28,12.33,87,127.58,0,...,11,6,9,1,1/9,9.00,9.00,6.00,0,0


## Dataset Overview

- Grain: One row per player per IPL season
- Coverage: Up to 2024 season
- Metrics: Batting, bowling, and fielding aggregates


In [20]:
expected_columns = {
    "Year", "Player_Name", "Matches_Batted", "Not_Outs",
    "Runs_Scored", "Highest_Score", "Batting_Average",
    "Balls_Faced", "Batting_Strike_Rate", "Centuries",
    "Half_Centuries", "Fours", "Sixes",
    "Catches_Taken", "Stumpings",
    "Matches_Bowled", "Balls_Bowled", "Runs_Conceded",
    "Wickets_Taken", "Best_Bowling_Match",
    "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls",
    "Five_Wicket_Hauls"
}

assert set(df_raw.columns) == expected_columns


In [21]:
df = df_raw.rename(columns=lambda x: x.lower())
df.columns = df.columns.str.replace(" ", "_")

In [23]:
df = df.rename(columns={
    "year": "season",
    "player_name": "player_name"
})

In [24]:
def generate_player_id(name):
    return hashlib.md5(name.lower().encode()).hexdigest()

df["player_id"] = df["player_name"].apply(generate_player_id)   
df.head()

,season,player_name,matches_batted,not_outs,runs_scored,highest_score,batting_average,balls_faced,batting_strike_rate,centuries,...,balls_bowled,runs_conceded,wickets_taken,best_bowling_match,bowling_average,economy_rate,bowling_strike_rate,four_wicket_hauls,five_wicket_hauls,player_id
0,NaN,Aaron Hardie,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,...,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,No stats,137c38bcb994c4cabc2238dd5a7b4bb4
1,2024.0,Abdul Samad,16,2,182,37*,18.2,108,168.52,0,...,0,0,0,0,0,0,0,0,0,7158f272ae5704cd1faea30ed8f103f9
2,2023.0,Abdul Samad,9,4,169,37*,42.25,128,132.03,0,...,0,0,0,0,0,0,0,0,0,7158f272ae5704cd1faea30ed8f103f9
3,2022.0,Abdul Samad,2,0,4,4,2.0,7,57.14,0,...,6,8,0,0/8,0,8.00,0,0,0,7158f272ae5704cd1faea30ed8f103f9
4,2021.0,Abdul Samad,11,1,111,28,12.33,87,127.58,0,...,6,9,1,1/9,9.00,9.00,6.00,0,0,7158f272ae5704cd1faea30ed8f103f9


In [25]:
numeric_cols = df.columns.difference(["player_name", "player_id"])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

In [26]:
df["data_source"] = DATA_SOURCE
df["ingestion_date"] = INGESTION_DATE

In [27]:
assert df.duplicated(subset=["player_id", "season"]).sum() == 0

## Metric Grouping

- Batting metrics
- Bowling metrics
- Fielding metrics


In [28]:
fact_player_season_stats = df.copy()
fact_player_season_stats.head()

,season,player_name,matches_batted,not_outs,runs_scored,highest_score,batting_average,balls_faced,batting_strike_rate,centuries,...,wickets_taken,best_bowling_match,bowling_average,economy_rate,bowling_strike_rate,four_wicket_hauls,five_wicket_hauls,player_id,data_source,ingestion_date
0,NaN,Aaron Hardie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137c38bcb994c4cabc2238dd5a7b4bb4,IPL Player Season Stats CSV,2026-02-09T00:39:48.054477
1,2024.0,Abdul Samad,16.0,2.0,182.0,NaN,18.20,108.0,168.52,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7158f272ae5704cd1faea30ed8f103f9,IPL Player Season Stats CSV,2026-02-09T00:39:48.054477
2,2023.0,Abdul Samad,9.0,4.0,169.0,NaN,42.25,128.0,132.03,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7158f272ae5704cd1faea30ed8f103f9,IPL Player Season Stats CSV,2026-02-09T00:39:48.054477
3,2022.0,Abdul Samad,2.0,0.0,4.0,4.0,2.00,7.0,57.14,0.0,...,0.0,NaN,0.0,8.0,0.0,0.0,0.0,7158f272ae5704cd1faea30ed8f103f9,IPL Player Season Stats CSV,2026-02-09T00:39:48.054477
4,2021.0,Abdul Samad,11.0,1.0,111.0,28.0,12.33,87.0,127.58,0.0,...,1.0,NaN,9.0,9.0,6.0,0.0,0.0,7158f272ae5704cd1faea30ed8f103f9,IPL Player Season Stats CSV,2026-02-09T00:39:48.054477


In [33]:
output_path = "../data/processed/player_stats_till_2024/fact_player_season_stats.parquet"
fact_player_season_stats.to_parquet(
    output_path,
    index=False,
    engine="pyarrow"
)

## Output

- Table: `fact_player_season_stats`
- Grain: Player–Season
- Usage:
  - Career trends
  - Baseline projections
  - Contextual fantasy analytics
